Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [48]:
# Hyperparameters
batch_size = 128
layer_size = [image_size * image_size, 1024, 10]
num_steps = 3000
l2_factor = 0.1 / (image_size * image_size)

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    
    weights = []
    biases = []
    for i in range(len(layer_size) - 1):
        weights.append(tf.Variable(tf.truncated_normal([layer_size[i], layer_size[i + 1]])))
        biases.append(tf.Variable(tf.zeros([layer_size[i + 1]])))
    
    def forward(x):
        layer0 = tf.nn.relu(tf.matmul(x, weights[0]) + biases[0])   
        return tf.matmul(layer0, weights[1]) + biases[1]
    
    logits = forward(tf_train_dataset)
    tf_l2_factor = tf.constant(l2_factor)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
            # (tf_l2_factor * tf.nn.l2_loss(weights[0])) + \          
            #(tf_l2_factor * tf.nn.l2_loss(weights[1]))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_pred = tf.nn.softmax(logits)
    valid_pred = tf.nn.softmax(forward(tf_valid_dataset))
    test_pred = tf.nn.softmax(forward(tf_test_dataset))
    
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        start = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[start : start + batch_size,:]
        batch_labels = train_labels[start : start + batch_size]
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_pred], feed_dict = feed_dict)
        if step % 500 == 0:
            print("Step %d microbatch loss %f" % (step, l))
            print("Microbatch test accuracy %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy %.1f%%" % accuracy(valid_pred.eval(), valid_labels))
    print("Test accuracy %.1f%%" % accuracy(test_pred.eval(), test_labels))
    
        
        
    

Initialized
Step 0 microbatch loss 285.479980
Microbatch test accuracy 8.6%
Validation accuracy 26.8%
Step 500 microbatch loss 10.859406
Microbatch test accuracy 75.8%
Validation accuracy 81.5%
Test accuracy 87.1%


In [49]:
batch_size = 128
layer_size = [image_size * image_size, 1024, 10]
num_steps = 1000
l2_factor = 0 / (image_size * image_size)

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    
    weights = []
    biases = []
    for i in range(len(layer_size) - 1):
        weights.append(tf.Variable(tf.truncated_normal([layer_size[i], layer_size[i + 1]])))
        biases.append(tf.Variable(tf.zeros([layer_size[i + 1]])))
    
    def forward(x):
        layer0 = tf.nn.relu(tf.matmul(x, weights[0]) + biases[0])   
        return tf.matmul(layer0, weights[1]) + biases[1]
    
    logits = forward(tf_train_dataset)
    tf_l2_factor = tf.constant(l2_factor)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
            # (tf_l2_factor * tf.nn.l2_loss(weights[0])) + \          
            #(tf_l2_factor * tf.nn.l2_loss(weights[1]))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_pred = tf.nn.softmax(logits)
    valid_pred = tf.nn.softmax(forward(tf_valid_dataset))
    test_pred = tf.nn.softmax(forward(tf_test_dataset))

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        start = (batch_size * 7)
        batch_data = train_dataset[start : start + batch_size,:]
        batch_labels = train_labels[start : start + batch_size]
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_pred], feed_dict = feed_dict)
        if step % 500 == 0:
            print("Step %d microbatch loss %f" % (step, l))
            print("Microbatch test accuracy %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy %.1f%%" % accuracy(valid_pred.eval(), valid_labels))
    print("Test accuracy %.1f%%" % accuracy(test_pred.eval(), test_labels))

Initialized
Step 0 microbatch loss 308.262512
Microbatch test accuracy 7.8%
Validation accuracy 28.8%
Step 500 microbatch loss 0.000000
Microbatch test accuracy 100.0%
Validation accuracy 63.2%
Test accuracy 68.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [55]:
batch_size = 128
layer_size = [image_size * image_size, 1024, 10]
num_steps = 3000
l2_factor = 0.1 / (image_size * image_size)

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    
    weights = []
    biases = []
    for i in range(len(layer_size) - 1):
        weights.append(tf.Variable(tf.truncated_normal([layer_size[i], layer_size[i + 1]])))
        biases.append(tf.Variable(tf.zeros([layer_size[i + 1]])))
    
    def forward_dropout(x):
        layer0 = tf.nn.dropout(tf.nn.relu(tf.matmul(x, weights[0]) + biases[0]),0.5) 
        return tf.nn.dropout(tf.matmul(layer0, weights[1]) + biases[1], 0.5)
    def forward(x):
        layer0 = tf.nn.relu(tf.matmul(x, weights[0]) + biases[0])   
        return tf.matmul(layer0, weights[1]) + biases[1]
    
    logits = forward_dropout(tf_train_dataset)
    tf_l2_factor = tf.constant(l2_factor)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    (tf_l2_factor * tf.nn.l2_loss(weights[0])) + \
    (tf_l2_factor * tf.nn.l2_loss(weights[1]))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_pred = tf.nn.softmax(forward(tf_train_dataset)
    valid_pred = tf.nn.softmax(forward(tf_valid_dataset))
    test_pred = tf.nn.softmax(forward(tf_test_dataset))

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        start = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[start : start + batch_size,:]
        batch_labels = train_labels[start : start + batch_size]
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        _, l, train_pred = session.run([optimizer, loss, train_pred], feed_dict = feed_dict)
        if step % 500 == 0:
            print("Step %d microbatch loss %f" % (step, l))
            print("Microbatch test accuracy %.1f%%" % accuracy(train_pred.eval(), batch_labels))
            print("Validation accuracy %.1f%%" % accuracy(valid_pred.eval(), valid_labels))
    print("Test accuracy %.1f%%" % accuracy(test_pred.eval(), test_labels))

Initialized
Step 0 microbatch loss 832.556580
Microbatch test accuracy 10.9%
Validation accuracy 18.6%
Step 500 microbatch loss 84.218231
Microbatch test accuracy 48.4%
Validation accuracy 79.9%
Step 1000 microbatch loss 70.427795
Microbatch test accuracy 42.2%
Validation accuracy 76.6%
Step 1500 microbatch loss 90.975639
Microbatch test accuracy 40.6%
Validation accuracy 75.8%
Step 2000 microbatch loss 44.950993
Microbatch test accuracy 36.7%
Validation accuracy 75.3%
Step 2500 microbatch loss 40.404484
Microbatch test accuracy 39.8%
Validation accuracy 74.8%
Test accuracy 81.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [8]:
encodeHiddenWidth = 500
encodeWidth = 100
hiddenWidth1 = 1024
hiddenWidth2 = 200
num_steps = 8000
num_steps_Auto = 800
batch_size = 128

learning_rate = 0.35
decay_rate = 0.9
decay_steps = 2000

loss_l2_factor = 0.00 / (image_size * image_size)

graph = tf.Graph()
with graph.as_default():
    # Combine tanh-activation autoencoder with 1-hidden-layer ReLU NN
    global_step = tf.Variable(0, trainable=False)
    #learning_rate = tf.train.exponential_decay(0.5, global_step)
    
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    
    encodeHiddenWeight = tf.Variable(tf.truncated_normal([image_size * image_size, encodeHiddenWidth]))
    encodeHiddenBias = tf.Variable(tf.zeros([encodeHiddenWidth]))
    encodeWeight = tf.Variable(tf.truncated_normal([encodeHiddenWidth, encodeWidth]))
    encodeBias = tf.Variable(tf.zeros([encodeWidth]))
    decodeWeight = tf.Variable(tf.truncated_normal([encodeWidth, image_size * image_size]))
    decodeBias = tf.Variable(tf.zeros([image_size * image_size]))
    
    def encode(x):
        hidden = tf.nn.tanh(tf.matmul(x, encodeHiddenWeight) + encodeHiddenBias)
        return tf.nn.tanh(tf.matmul(hidden, encodeWeight) + encodeBias)
    
    def autoencode(x):
        return tf.matmul(encode(x), decodeWeight) + decodeBias
    
    def magnitude(x):
        return tf.reduce_sum(tf.mul(x,x))
    lossAuto = magnitude((autoencode(tf_train_dataset) - tf_train_dataset)) / magnitude(tf_train_dataset)
    optimizerAuto = tf.train.GradientDescentOptimizer(0.1).minimize(lossAuto)
    
    weight0 = tf.Variable(tf.truncated_normal([image_size * image_size, hiddenWidth1]))
    bias0 = tf.Variable(tf.zeros([hiddenWidth1]))
    weight1 = tf.Variable(tf.truncated_normal([hiddenWidth1, hiddenWidth2]))
    bias1 = tf.Variable(tf.zeros([hiddenWidth2]))
    weight2 = tf.Variable(tf.truncated_normal([hiddenWidth2 + encodeWidth, num_labels]))
    bias2 = tf.Variable(tf.zeros([num_labels]))
    
    def forward(x):
        layer0 = tf.nn.tanh(tf.matmul(x, weight0) + bias0)
        layer1 = tf.nn.tanh(tf.matmul(layer0, weight1) + bias1)
        return tf.matmul(tf.concat(1, [layer1, encode(x)]), weight2) + bias2
    
    def forward_dropout(x):
        layer0 = tf.nn.tanh(tf.nn.dropout(tf.matmul(x, weight0) + bias0, 0.5))
        layer1 = tf.nn.tanh(tf.nn.dropout(tf.matmul(layer0, weight1) + bias1, 0.5))
        return tf.matmul(tf.concat(1, [layer1, encode(x)]), weight2) + bias2
    
    def my_cross_entropy(logits,labels):
        prob = tf.nn.softmax(logits)
        return tf.reduce_mean(tf.mul(labels, tf.log(prob + 10 ** (-30))))
    
    loss = (-1 * my_cross_entropy(forward_dropout(tf_train_dataset), tf_train_labels))
    # loss_l2_factor * (tf.nn.l2_loss(weight0) + tf.nn.l2_loss(weight1) + tf.nn.l2_loss(weight2))
    rate = tf.train.exponential_decay(learning_rate, global_step, decay_steps, decay_rate)
    optimizer = tf.train.GradientDescentOptimizer(rate).minimize(loss, global_step=global_step)
    
    train_pred = tf.nn.softmax(forward(tf_train_dataset))
    valid_pred = tf.nn.softmax(forward(tf_valid_dataset))
    test_pred = tf.nn.softmax(forward(tf_test_dataset))
print("Graph built")

Graph built


In [9]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    tf.train.global_step(session, global_step)
    print("Initialized")
    print("Starting autoencoder training")
    for step in range(num_steps_Auto):
        start = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[start:(start + batch_size),:]
        batch_labels = train_labels[start:(start + batch_size),:]
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        _, lA = session.run([optimizerAuto, lossAuto], 
                            feed_dict = feed_dict)
        if step % 100 == 0:
            print("Step %d, Loss %f" % (step, lA))
    print("Finished autoencoder training")
    print("Beginning neural net training")
    for step in range(num_steps):
        #permute list
        #pull things from list in chunks of size batch_size
        start = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[start:(start + batch_size),:]
        batch_labels = train_labels[start:(start + batch_size),:]
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_pred], 
                                        feed_dict = feed_dict)
        if step % 100 == 0:
            print("%d: Train accuracy %.1f%%, loss: %f" % (step, accuracy(predictions, batch_labels), l))
            print("Validation accuracy %.1f%%" % accuracy(valid_pred.eval(), valid_labels))
    print("Test accuracy %.1f%%, loss: %f" % (accuracy(test_pred.eval(), test_labels), l))
    print("Done")

Initialized
Starting autoencoder training
Step 0, Loss 357.678741
Step 100, Loss 28.598131
Step 200, Loss 32.562088
Step 300, Loss 17.582573
Step 400, Loss 22.706758
Step 500, Loss 13.879094
Step 600, Loss 6.125193
Step 700, Loss 13.355504
Finished autoencoder training
Beginning neural net training
0: Train accuracy 13.3%, loss: 1.695542
Validation accuracy 12.4%
100: Train accuracy 21.9%, loss: 0.965698
Validation accuracy 23.6%
200: Train accuracy 33.6%, loss: 1.095282
Validation accuracy 35.4%
300: Train accuracy 43.8%, loss: 0.807944
Validation accuracy 45.6%
400: Train accuracy 57.8%, loss: 0.770340
Validation accuracy 52.9%
500: Train accuracy 54.7%, loss: 0.711360
Validation accuracy 57.8%
600: Train accuracy 56.2%, loss: 0.657331
Validation accuracy 60.8%
700: Train accuracy 71.9%, loss: 0.578146
Validation accuracy 63.7%
800: Train accuracy 68.8%, loss: 0.519240
Validation accuracy 65.2%
900: Train accuracy 62.5%, loss: 0.552476
Validation accuracy 67.4%
1000: Train accuracy 6